## LDA

In [2]:
# necessary imports
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from pprint import pprint
import pandas as pd
from nltk.corpus import stopwords
import re

In [3]:
# data
path_plot = 'data/plot_summaries.txt'
plot_summary_df = pd.read_csv(path_plot, delimiter='\t', header=None)
plot_summary_df.columns = ['movie_id', 'plot_summary']

In [5]:
# Create a dataframe with one cell resulting of the concatenation of the summaries
all_summaries = plot_summary_df['plot_summary'].str.cat(sep=' ')

concat_summaries_df = pd.DataFrame({'all_summaries': [all_summaries]})
print(concat_summaries_df)

                                       all_summaries
0  Shlykov, a hard-working taxi driver and Lyosha...


Code below inspired from : https://medium.com/@pinakdatta/understanding-lda-unveiling-hidden-topics-in-text-data-9bbbd25ae162

In [32]:
def text_split(text) :
    
    #initializaing a set of frequent and not relevent words in english
    stop_words = set(stopwords.words('english'))

    #Remove from text all punctuations
    text = re.sub(r'[^\w\s]', '', text)
    
    #we transform all the strings into minuscules using lower function + and split them into a list of word (under string format)
    #we only keep the words that don't appear in the stop_words set
    text_splited = [word for word in text.lower().split() if word not in stop_words]
    
    return text_splited

def bag_of_words(text_splited) : 
    
    # Create a dictionary mapping each word to a unique id
    dictionary = Dictionary([text_splited])
    
    
    corpus = dictionary.doc2bow(text_splited)
    
    return corpus, dictionary
    
def lda(df_line) : 
    corpus, dictionary = df_line['corpus']
    
    lda_model = LdaModel([corpus], num_topics=1, id2word=dictionary)
    return lda_model

def get_topics(lda_model):
    
    return lda_model.show_topics(num_topics=1, num_words=5, formatted=True)

In [19]:
# We transforme the type of the column plot_summary (str) into lists of str and split the words composing it
plot_summary_df['split'] = plot_summary_df['plot_summary'].apply(text_split)


In [20]:
#we convert the splited lists into bag-of-words
plot_summary_df['corpus'] = plot_summary_df['split'].apply(bag_of_words)

In [21]:
# Train the LDA model
plot_summary_df['lda'] = plot_summary_df.apply(lda, axis=1)

In [33]:
pprint(plot_summary_df['lda'].apply(get_topics).tolist())

Grouper les résumés par décennie.

In [36]:
# We transforme the type of the column all_summaries (str) into lists of str and split the words composing it
concat_summaries_df['split'] = concat_summaries_df['all_summaries'].apply(text_split)


In [37]:
#we convert the splited lists into bag-of-words
concat_summaries_df['corpus'] = concat_summaries_df['split'].apply(bag_of_words)

In [38]:
# Train the LDA model
concat_summaries_df['lda'] = concat_summaries_df.apply(lda, axis=1)

In [39]:
pprint(concat_summaries_df['lda'].apply(get_topics).tolist())

[[(0,
   '0.004*"one" + 0.003*"back" + 0.003*"two" + 0.003*"film" + 0.003*"tells"')]]
